# Entrenamiento de los modelos

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import numpy as np

In [ ]:
dfPath = "dataframe/train_df.csv"
train_df = np.loadtxt(dfPath, delimiter=',', skiprows=1)

In [ ]:
# 1. Obtener IDs únicos
unique_ids = train_df['id'].unique()
print(f"Total de documentos únicos: {len(unique_ids)}")

# 2. Crear mapping de ID a clase
id_to_class = train_df.groupby('id')['is_human'].first().to_dict()

# 3. Split de IDs (no de oraciones)
from sklearn.model_selection import train_test_split

# Preparar listas para stratified split
ids_list = list(id_to_class.keys())
labels_list = [id_to_class[id_] for id_ in ids_list]

train_ids, test_ids = train_test_split(
    ids_list, 
    test_size=0.2, 
    random_state=42, 
    stratify=labels_list  # Mantener proporción de clases
)

print(f"Documentos en train: {len(train_ids)}")
print(f"Documentos en test: {len(test_ids)}")

# 4. Filtrar oraciones según los IDs
train_sentences = train_df[train_df['id'].isin(train_ids)]
test_sentences = train_df[train_df['id'].isin(test_ids)]

print(f"\nOraciones en train: {len(train_sentences)}")
print(f"Oraciones en test: {len(test_sentences)}")

# 5. Preparar X e y
X_train_doc = train_sentences[feature_columns].values
y_train_doc = train_sentences['is_human'].values
X_test_doc = test_sentences[feature_columns].values
y_test_doc = test_sentences['is_human'].values

# 6. Entrenar modelo
print("\n" + "="*50)
print("Entrenando SVM con split a nivel de documento...")
print("="*50)

svm_model_doc = SVC(kernel='rbf', C=1.0, gamma='scale', random_state=42)
svm_model_doc.fit(X_train_doc, y_train_doc)

# 7. Evaluación
y_pred_doc = svm_model_doc.predict(X_test_doc)

print("\nResultados en Test Set (Split por Documento):")
print(f"Accuracy: {accuracy_score(y_test_doc, y_pred_doc):.4f}")
print("\nClassification Report:")
print(classification_report(y_test_doc, y_pred_doc, target_names=['IA', 'Humano']))
print("\nConfusion Matrix:")
print(confusion_matrix(y_test_doc, y_pred_doc))

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices